### Capstone Final Project – The Battle of Neighborhoods

#### Find better place in Toronto,Canada

Objective : For any tourist who are visiting Toronto , this project will help in anaysis of the place with all facilities

### 1.Importing and installing Python Libraries and Dependencies

In [1]:
# package to convert address to coordinates
!pip install geocoder
# visualize data into Leaflet map
!pip install folium

     |████████████████████████████████| 102kB 4.9MB/s ta 0:00:011
     |████████████████████████████████| 829kB 6.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


## Importing Libraries 

In [2]:
import pandas as pd  # data processing
import numpy as np # linear algebra

#visualization
import requests # library to hanle request

# To see full dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json #libraries to handle JSON files
from pandas.io.json import json_normalize #transform JSON files into pandas datframe

In [3]:
pip install geopy

     |████████████████████████████████| 112kB 4.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Convert an address into latitude and longitude values
import geocoder
from geopy.geocoders import Nominatim

In [5]:
#Pull data from HTML file
!pip install bs4

In [6]:
from bs4 import BeautifulSoup

In [7]:
#Matplotlib and associated plotting modules for data visualization and exploratory data analysis
import matplotlib as mpl
import matplotlib.pyplot as plt

#check latest version of matplotlib
print('Matplotlib version: ', mpl.__version__)

Matplotlib version:  3.3.0


In [8]:
#matplot lib and associated plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2.Data extraction and cleaning

##### Extracting Postal codes from the given wikipedia-https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [9]:
Postalurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(Postalurl)
soup = BeautifulSoup(extracting_data.content, 'html5lib')

#### Converting content of PostalCodes HTML table as dataframe

In [10]:
toronto  = pd.DataFrame(columns ={'Postalcode','Borough','Neighborhood'})

wiki_table = soup.find('div', attrs = {'class':'mw-parser-output'}) 
table_rows = wiki_table.table.tbody

postalcode = 0
borough = 0
neighborhood = 0

for tr in table_rows.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
       if i == 0:        
           postalcode = td.text.strip('\n')
           i = i + 1
       elif i == 1:
           borough = td.text.strip('\n')
           i = i + 1
       elif i == 2:
           neighborhood = td.text.strip('\n')
    toronto=toronto.append({'Postalcode':postalcode,'Borough': borough,'Neighborhood':neighborhood}, ignore_index=True)
toronto.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


#### Data cleaning of postal_code

In [11]:
#removing non-assigned and 0 values rows
toronto = toronto[toronto.Borough!=0]
empty = 'Not assigned'
toronto = toronto[toronto.Borough!=empty]
toronto = toronto[(toronto.Postalcode!=empty) & (toronto.Borough!= empty) & (toronto.Neighborhood!=empty)]
toronto.head()

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#Rearranging indexes
toronto.reset_index(drop = True, inplace= True)
toronto.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
toronto.shape

(103, 3)

In [14]:
#Merging rows with same postalcode
df_toronto = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df_toronto.shape

(103, 3)

In [16]:
df_toronto.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4G,North York,Downsview
freq,1,24,4


#### saving the file for future use

In [17]:
df_toronto.to_csv('toronto_postal.csv')

#### Define location based on pincode

In [18]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords= g.latlng
    latitude = lati_long_coords[0]
    longitude = lati_long_coords[1]
    return postal_code,latitude,longitude

#### Retreiving postal co-ordinates of toronto Postalcode

In [19]:
#Return longtitude & latitude of each postal codes of toronto
postal_codes = df_toronto['Postalcode']
coords_list = []
for postal_code in postal_codes.tolist():
    coords = get_latilong(postal_code)
    coords_list.append(coords)
print(coords_list)

[('M1B', 43.81153000000006, -79.19551999999999), ('M1C', 43.78564000000006, -79.15870999999999), ('M1E', 43.765750000000025, -79.17519999999996), ('M1G', 43.768200000000036, -79.21760999999998), ('M1H', 43.769690000000026, -79.23943999999995), ('M1J', 43.74309000000005, -79.23525999999998), ('M1K', 43.72861000000006, -79.26366999999993), ('M1L', 43.714060000000075, -79.28411999999997), ('M1M', 43.72360000000003, -79.23495999999994), ('M1N', 43.69539000000003, -79.26193999999998), ('M1P', 43.75998000000004, -79.26836999999995), ('M1R', 43.750710000000026, -79.30055999999996), ('M1S', 43.79394000000008, -79.26710999999995), ('M1T', 43.784730000000025, -79.29936999999995), ('M1V', 43.817810000000065, -79.28023999999994), ('M1W', 43.80052000000006, -79.32073999999994), ('M1X', 43.83422000000007, -79.21669999999995), ('M2H', 43.802850000000035, -79.35620999999998), ('M2J', 43.780970000000025, -79.34780999999998), ('M2K', 43.78102000000007, -79.38059999999996), ('M2L', 43.757220000000075, -7

#### Adding location coordinates with each pincode

In [20]:
#Convert list to dataframe
df_coords= pd.DataFrame(coords_list, columns=['Postalcode','Latitude', 'Longitude'])
df_coords.head()

,Postalcode,Latitude,Longitude
0,M1B,43.81153,-79.19552
1,M1C,43.78564,-79.15871
2,M1E,43.76575,-79.17520
3,M1G,43.76820,-79.21761
4,M1H,43.76969,-79.23944


In [21]:
#Merging two tables one with neighhood deatils and other one with geoloactions
df_toronto_info = pd.merge(df_toronto, df_coords, on = 'Postalcode')
df_toronto_info.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


### Details of Downtown, Toronto

In [22]:
df_downtown_info = df_toronto_info[df_toronto_info.Borough == 'Downtown Toronto']
df_downtown_info.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.68190,-79.37850
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649
52,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38130
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818


In [23]:
#Rearranging table index
df_downtown_info.reset_index(inplace = True)
df_downtown_info.head()

,index,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,50,M4W,Downtown Toronto,Rosedale,43.68190,-79.37850
1,51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649
2,52,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38130
3,53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
4,54,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818


In [24]:
df_downtown_info.shape

(19, 6)

## Map of Downtown Toronto (Visualization)

In [25]:
address = 'Downtown,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geographical location of downtown toronto are {}, {}.'.format(latitude_x,longitude_y))

The geographical location of downtown toronto are 43.6541737, -79.38081164513409.


In [26]:
#create map of Downtown Toronto using latitude and longitude values
import folium
map_downtown = folium.Map(location=[latitude_x, longitude_y], zoom_start = 10)
map_downtown
#for lat, lng, nei in zip(df_downtown_info[''])                          

In [27]:
#add markers of Downtown to map
for lat,lng,nei in zip(df_downtown_info['Latitude'],df_downtown_info['Longitude'],df_downtown_info['Neighborhood']):
    label = '{}'.format(nei)
    # folium leaf for each loaction of downtown
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_downtown)
    
map_downtown

### FourSquare API

In [28]:
# FourSquare credential
CLIENT_ID = 'LQGTFX03OHVQ3WEZKI2LIC3DVUOIKIXF03Z0WL5F3SHZ1LPD'
CLIENT_SECRET = 'HLCO0K11A1B5ZANAJ4VADB4AVPFGX332RASHI51GYLUFV3SI' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LQGTFX03OHVQ3WEZKI2LIC3DVUOIKIXF03Z0WL5F3SHZ1LPD
CLIENT_SECRET:HLCO0K11A1B5ZANAJ4VADB4AVPFGX332RASHI51GYLUFV3SI


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
                    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
#venues near DownTown
Downtown_venues = getNearbyVenues(names = df_downtown_info['Neighborhood'],
                                    latitudes = df_downtown_info['Latitude'],
                                    longitudes = df_downtown_info['Longitude'])

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [31]:
print(Downtown_venues.shape)
Downtown_venues.head()

(1250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.6819,-79.3785,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
1,Rosedale,43.6819,-79.3785,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.6819,-79.3785,Betline Trail at Roxborough dr.,43.680530,-79.381490,Bike Trail
3,Rosedale,43.6819,-79.3785,Whitney Park,43.682036,-79.373788,Park
4,Rosedale,43.6819,-79.3785,Petite & Sweet,43.685982,-79.376072,Shop & Service


In [32]:
print('There are {} Uniques Categories.'.format(len(Downtown_venues['Venue Category'].unique())))
Downtown_venues.groupby('Neighborhood').count()

There are 192 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,67,67,67,67,67,67
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",71,71,71,71,71,71
Central Bay Street,74,74,74,74,74,74
Christie,11,11,11,11,11,11
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",52,52,52,52,52,52


### One Hot Encoding of Features

In [33]:
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix = "", prefix_sep = "")
Downtown_onehot.head(2)

,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
#add neighborhood column back to dataframe
Downtown_onehot['Neighborhood']=Downtown_venues['Neighborhood']

#move neighborhood column to the first column(here -1 is shifting 1 step back)
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]
Downtown_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
Downtown_onehot.shape

(1250, 192)

In [36]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.014925,0.0,0.000000,0.000000,0.0,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.014925,0.029851,0.000000,0.000000,0.014925,0.029851,0.0,0.0,0.0,0.014925,0.0,0.000000,0.0,0.029851,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.029851,0.0,0.000000,0.000000,0.029851,0.0,0.0,0.014925,0.029851,0.104478,0.0,0.000000,0.0,0.0,0.0,0.014925,0.000000,0.014925,0.0,0.000000,0.0,0.014925,0.000000,0.0,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.0,0.014925,0.000000,0.000000,0.0,0.0,0.0,0.029851,0.000000,0.000000,0.014925,0.000000,0.0,0.014925,0.014925,0.014925,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.014925,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.0,0.0,0.029851,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.029851,0.014925,0.014925,0.0,0.000000,0.0,0.000000,0.0,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.0,0.0,0.014925,0.014925,0.0,0.0,0.000000,0.000000,0.014925,0.0,0.000000,0.0,0.0,0.0,0.0,0.014925,0.0,0.0,0.000000,0.0,0.014925,0.000000,0.000000,0.000000,0.000000,0.0,0.014925,0.000000,0.0,0.044776,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.0,0.014925,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.014925,0.000000,0.0,0.0149

### Most common venues near neighborhood

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Café,Cheese Shop,Breakfast Spot,Seafood Restaurant,Cocktail Bar,Beer Bar,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Speakeasy,Restaurant,Bakery,Park,Intersection,Electronics Store
2,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Plaza,Sushi Restaurant,Italian Restaurant,Sandwich Place,Electronics Store,Cosmetics Shop
3,Christie,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Distribution Center,Event Space
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Bubble Tea Shop,Dance Studio,Café,Men's Store,Hotel


### CLUSTERING NEIGHBORHOODS

#### Now we applied Machine Learning Technique “Clustering” to segment the neighborhoods in similar objects cluster.

### K-Means Clustering Approach

In [39]:
# Using K-Means to cluster neighborhood into 3 clusters
Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)
Downtown_grouped_clustering

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.014925,0.029851,0.000000,0.000000,0.014925,0.029851,0.000000,0.000000,0.0,0.014925,0.000000,0.000000,0.00,0.029851,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.029851,0.000000,0.000000,0.000000,0.029851,0.000000,0.00,0.014925,0.029851,0.104478,0.000000,0.000000,0.00,0.000000,0.00,0.014925,0.000000,0.014925,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.000000,0.014925,0.000000,0.00,0.014925,0.014925,0.014925,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.014925,0.014925,0.000000,0.014925,0.000000,0.000000,0.000000,0.00,0.000000,0.029851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029851,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.00,0.000000,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.00,0.014925,0.000000,0.0

In [40]:
#Here number of elemt in array is number of rows in downtown group)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3, random_state=0).fit(Downtown_grouped_clustering)
kmeans.labels_

array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)

In [41]:
#Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
#Run below code if cluster column is already there
#neighborhoods_venues_sorted.drop(columns= 'Cluster Labels' , inplace = True)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Coffee Shop,Restaurant,Farmers Market,Café,Cheese Shop,Breakfast Spot,Seafood Restaurant,Cocktail Bar,Beer Bar,Italian Restaurant
1,1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Speakeasy,Restaurant,Bakery,Park,Intersection,Electronics Store
2,1,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Plaza,Sushi Restaurant,Italian Restaurant,Sandwich Place,Electronics Store,Cosmetics Shop
3,2,Christie,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Distribution Center,Event Space
4,1,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Bubble Tea Shop,Dance Studio,Café,Men's Store,Hotel


In [42]:
df_downtown_info.head(2)

,index,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,50,M4W,Downtown Toronto,Rosedale,43.68190,-79.37850
1,51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649


In [43]:
#Merge downtown_merge(contains geoloaction) with neighborhoods_venues_sorted(contains neighborhood common values)
#both table has common neighborhood which set as index
Downtown_merged = df_downtown_info
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
Downtown_merged.head()

,index,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,50,M4W,Downtown Toronto,Rosedale,43.68190,-79.37850,0,Park,Shop & Service,Bike Trail,Playground,Tennis Court,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66788,-79.36649,1,Café,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Bakery,Outdoor Sculpture,Convenience Store
2,52,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38130,1,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Bubble Tea Shop,Dance Studio,Café,Men's Store,Hotel
3,53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
4,54,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Bookstore,Diner,Bubble Tea Shop


### Map of Clusters (visualization)

In [44]:
kclusters = 10

#set color scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print('ys :', ys)

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
print('colors_array: ' ,colors_array)

rainbow = [colors.rgb2hex(i) for i in colors_array]
print('rainbow:' ,rainbow)

ys : [array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 1,  3,  7, 13, 21, 31, 43, 57, 73, 91]), array([  2,   7,  20,  41,  70, 107, 152, 205, 266, 335]), array([  3,  13,  41,  87, 151, 233, 333, 451, 587, 741]), array([   4,   21,   70,  151,  264,  409,  586,  795, 1036, 1309]), array([   5,   31,  107,  233,  409,  635,  911, 1237, 1613, 2039]), array([   6,   43,  152,  333,  586,  911, 1308, 1777, 2318, 2931]), array([   7,   57,  205,  451,  795, 1237, 1777, 2415, 3151, 3985]), array([   8,   73,  266,  587, 1036, 1613, 2318, 3151, 4112, 5201]), array([   9,   91,  335,  741, 1309, 2039, 2931, 3985, 5201, 6579])]
colors_array:  [[5.00000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [2.80392157e-01 3.38158275e-01 9.85162233e-01 1.00000000e+00]
 [6.07843137e-02 6.36474236e-01 9.41089253e-01 1.00000000e+00]
 [1.66666667e-01 8.66025404e-01 8.66025404e-01 1.00000000e+00]
 [3.86274510e-01 9.84086337e-01 7.67362681e-01 1.00000000e+00]
 [6.13725490e-01 9.84086337e-01 6.41213315e-

In [45]:
#create map
map_clusters = folium.Map(location = [latitude_x, longitude_y], zoom_start = 11)

#add marker to map
marker_colors = []
for lat, lon, nei, cluster in zip(Downtown_merged['Latitude'],
                                  Downtown_merged['Longitude'],
                                  Downtown_merged['Neighborhood'],
                                  Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + 'cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7
        ).add_to(map_clusters)

map_clusters

### Examing Clusters

#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 0

In [46]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, 
                    Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Postalcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,-79.3785,0,Park,Shop & Service,Bike Trail,Playground,Tennis Court,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 1

In [47]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, 
                    Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Postalcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4X,-79.36649,1,Café,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Bakery,Outdoor Sculpture,Convenience Store
2,M4Y,-79.38130,1,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Bubble Tea Shop,Dance Studio,Café,Men's Store,Hotel
3,M5A,-79.36265,1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
4,M5B,-79.37818,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Bookstore,Diner,Bubble Tea Shop
5,M5C,-79.37557,1,Seafood Restaurant,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Lingerie Store,Diner
6,M5E,-79.37368,1,Coffee Shop,Restaurant,Farmers Market,Café,Cheese Shop,Breakfast Spot,Seafood Restaurant,Cocktail Bar,Beer Bar,Italian Restaurant
7,M5G,-79.38493,1,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Plaza,Sushi Restaurant,Italian Restaurant,Sandwich Place,Electronics Store,Cosmetics Shop
8,M5H,-79.38258,1,Hotel,Coffee Shop,Café,Japanese Restaurant,Gym,Asian Restaurant,Steakhouse,American Restaurant,Restaurant,Salad Place
9,M5J,-79.38040,1,Coffee Shop,Hotel,Japanese Restaurant,Boat or Ferry,Park,Aquarium,Deli / Bodega,Plaza,Shopping Mall,Basketball Stadium
10,M5K,-79.38153,1,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Salad Place,Asian Restaurant,Beer Bar


#### Cluster 2

In [48]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, 
                    Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Postalcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M6G,-79.42071,2,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Distribution Center,Event Space


Conclusion : Based on the analysis of the clusters , we came to conclusion that cluster 1 have most
to Cafe, School, Super market, medical shops, grocery shops, mall, theatre, hospital. This place is ideal for any tourist who visit Toronto with all facilities.
